# Load Data

> Functions that load the data for the map.   

**Contents**

`Solomon Geospatial Data`
- `SolomonGeo`: A class that cleans the solomon islandas census and geography data
- `SolomonGeo.read_test`: Loads and transforms the test data
- `SolomonGeo.get_geojson`: Returns the geo_df as a geojson datatset

In [ ]:
#| default_exp load_data

In [ ]:
#| export
from nbdev.showdoc import *
import geopandas as gpd
import pandas as pd
from git import Repo
import json
from fastcore import *
from fastcore.basics import patch
from fastcore.test import *
import sys
import topojson as tp
import pickle

In [ ]:
#| hide
repo = Repo('.', search_parent_directories=True)
fp = str(repo.working_tree_dir) + "/testData/"
const_df = pd.read_csv(fp + 'sol_census_' + 'all' + '_' + '2009' + '.csv')
# Check that the files exist using fastcore (both census and geo)

## Solomon Geospatial Data
> Load the geography and census data
### Geography Data
Solomons islands geography data is organised at the levels
 - adm0 - The country as as whole, Solomon Islands
 - adm1 - Also referred to as the province e.g. Honiara, Malaita
 - adm2 - The Consituency e.g. Central Honiara
 - adm3 - Ward, the smallest geography I am reporting. E.g. Cruz

### Census Data
 Solomon islands census data has been used from the 2009 and 2019 census. For the respective census:
2009
 - We have the total population in for each of the administration regions
2019
 - There is only data available down to the province level

In [ ]:
#| export
class SolomonGeo:
    # TODO work out how to format the attributes
    # Look at nbdev docs maybe?
    # TODO change all data to int?
    # TODO - should I make this a dataclass for the auto functionaliy? potentially should try it out
    '''
    Load the solomon islands geography data 
    Attributes:
        geo_df    Geopandas dataframe containing geographies and census data
        geo_levels    A list of the types of available aggregations
        census_vars    A list of census variables in the dataset 
        data_type   Specifies whether the variable is a percentage or number
        locations A dictionary of locations accessed by the geography level
    '''
    def __init__(self, 
                geo_df:gpd.GeoDataFrame): # A geopandas dataset containing population and geography boundaries for each aggregation
        self.geo_df = geo_df

        # variable that tracks the types of aggregations
        self.geo_levels = geo_df.loc[:, 'agg'].unique()

        # Save a list of census variables
        # TODO this list is repeated twice, how can I undo this harcoding??
        col_ignore = ['geometry', 'id', 'agg', 'year', 'type']
        self.census_vars = list(geo_df.drop(columns = col_ignore).columns)
        # TODO should captialise first letter
        self.data_type = geo_df.loc[:, 'type'].unique()

        # save a list of locations as a dictionary access by geography level
        locations = {}
        for geo in self.geo_levels:
            locations[geo] = geo_df.loc[geo_df['agg'] == geo].index.unique().sort_values()
        self.locations = locations
        # TODO: need a list of column sub headings: get from column name split by `:`

        self.type_default = 'Total'


    @classmethod
    def read_test(cls,
                 ): # A solmon geo class TODO work out how to return self here... (can't?)
        '''
        Initialise the object using the local testing data
        '''
        # TODO might need to further abstract this concatenation process
        df, geo = cls.extract_from_file('2009')

        gdf = cls.transform('2009', df, geo)

        return cls(
            geo_df = gdf
        )

    @classmethod
    def extract_from_file(cls, 
                            year:str, # The year of that data, only relevant for census data
                 ) -> (pd.DataFrame, 
                      gpd.GeoDataFrame): # Returns input pandas and geopandas datasets
        '''
        Extract and return input datasets from file. Assumes correct format of input dataset
        # TODO specify characteristics of dataset
        # TODO - refact load, too many of the cleaning functions are being done during the loading
        '''
        repo = Repo('.', search_parent_directories=True)
        pw = str(repo.working_tree_dir) + "/testData/"
        df = pd.read_csv(pw + 'sol_census_all_' + year + '.csv')
        aggs = df.loc[:, 'agg'].unique()
        geos = gpd.GeoDataFrame()
        for agg in aggs:
            geo = gpd.read_file(pw + 'sol_geo_' + agg.lower() + '.json')
            # Before combining, need to rename like columns
            # Rename columns and keep only necessary ones, Note that id can be province id, contsituency id etc.
            geo.columns = geo.columns.str.replace(r'^[a-zA-Z]+name$', 'geo_name', case = False, regex = True)
            # TODO this assumes the id key column is the first one (which so far it is...)
            geo.rename(columns = {geo.columns[0]:'id'}, inplace=True)

            # keep only the id and geometry columns
            geo = geo.loc[:, ['id', 'geometry']] 

            # Add an agg column, as the data and geometry need to be joined by id and agg
            geo.loc[:, 'agg'] = agg

            # simplify the geography, use topo to preserver the topology between shapes
            topo = tp.Topology(geo, prequantize=False)
            geo = topo.toposimplify(360/43200).to_gdf()

            geos = pd.concat([geos, geo])

        return (
            df, 
            geos
        )

    @classmethod
    def transform(cls, 
            year:str, # The year of that data, only relevant for census data
            df:pd.DataFrame, # Uncleaned input census dataset
            geo:gpd.GeoDataFrame, # Uncleaned input geospatial dataset
           )-> gpd.GeoDataFrame: # The geopandas dataset for given aggregation
        '''
        Tranform given raw input dataset into a cleaned and combined geopandas dataframe
        '''
        # Clean the geospatial dataframe
        # Add a column that indicates the year
        geo.loc[:, 'year'] = year
        
        # Clean the census data
        df = df.dropna()
        # Rename columns to be consistent across geography
        df.columns = df.columns.str.replace(r'^[a-zA-Z]+_name$', 'geo_name', case = False, regex = True)
        # id needs to change types twice so that it is a string of an int
        df = df.astype({'id': 'int'})#, 'male_pop':'int', 	'female_pop':'int', 'total_pop':'int'})
        df = df.astype({'id': 'str'})
        
        # Merge the data together
        geo_df = geo.merge(df, on=['id', 'agg']).set_index("geo_name") # , 'geo_name'
        return geo_df

    @classmethod
    def load_pickle(cls,
                    folder:str, #file path of the folder to save in
                    file_name:str = 'sol_geo.pickle' # file name of the saved class
                 ):
        '''
        Initialise the object from a saved filepath
        '''
        # TODO work out how to make this a class method
        repo = Repo('.', search_parent_directories=True)
        pw = str(repo.working_tree_dir) + folder + file_name
        
        with open(pw, 'rb') as f:
            tmp_geo = pickle.load(f)

        # TODO  guide said do below line, don't think relevant though
        #cls.__dict__.update(tmp_dict) 
        
        return cls(
            geo_df = gpd.GeoDataFrame(tmp_geo['geo_df'])
        )
        

In [ ]:
#| hide
show_doc(SolomonGeo)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L20){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo

>      SolomonGeo (geo_df:geopandas.geodataframe.GeoDataFrame)

Load the solomon islands geography data 
Attributes:
    geo_df    Geopandas dataframe containing geographies and census data
    geo_levels    A list of the types of available aggregations
    census_vars    A list of census variables in the dataset 
    data_type   Specifies whether the variable is a percentage or number
    locations A dictionary of locations accessed by the geography level

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_df | GeoDataFrame | A geopandas dataset containing population and geography boundaries for each aggregation |

In [ ]:
#| hide
show_doc(SolomonGeo.read_test)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L59){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.read_test

>      SolomonGeo.read_test ()

Initialise the object using the local testing data

## Save and Load

### Save Solomon Geo
Saves a solmon geo class in a pickle

In [ ]:
#| export
@patch
def save_pickle(self:SolomonGeo,
              folder:str, #file path of the folder to save in
                file_name:str = 'sol_geo.pickle' # file name of the saved class
             ):
    '''
    Save a pickle of the SolomonGeo class
    '''
    repo = Repo('.', search_parent_directories=True)
    pw = str(repo.working_tree_dir) + folder + file_name
    
    f = open(pw, 'wb')
    pickle.dump(self.__dict__, f, 2)
    f.close()

    # For now I will also save the goegraphy in an assets folder
    # TODO update this process in future - may need to save elsewhere
    # TODO I think I need to save in multiple spots
    pw_asset = str(repo.working_tree_dir) + "/assets/sol_geo.json"
    with open(pw_asset, 'w') as f:
        json.dump(self.get_geojson(geo_filter = 'ward'), f)


In [ ]:
show_doc(SolomonGeo.save_pickle)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L163){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.save_pickle

>      SolomonGeo.save_pickle (folder:str, file_name:str='sol_geo.pickle')

Save a pickle of the SolomonGeo class

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| folder | str |  | file path of the folder to save in |
| file_name | str | sol_geo.pickle | file name of the saved class |

In [ ]:
show_doc(SolomonGeo.load_pickle)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L139){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.load_pickle

>      SolomonGeo.load_pickle (folder:str, file_name:str='sol_geo.pickle')

Initialise the object from a saved filepath

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| folder | str |  | file path of the folder to save in |
| file_name | str | sol_geo.pickle | file name of the saved class |

## Get Geo JSON
A getter method for the geometry portion of the dataset that returns a geoJson formated geography. 

It only includes the geography and location name as id

In [ ]:
#| export
@patch
def get_geojson(self:SolomonGeo, 
                geo_filter:str = None, # Filters the geojson to the requested aggregation 
               ) -> dict: # Geo JSON formatted dataset
    '''
    A getter method for the SolomonGeo class that returns a Geo JSON formatted dataset
    '''
    ret = self.geo_df
    # Only need geojson from one half of the dataset
    ret = ret.loc[ret['type'] == self.type_default, :]
    if geo_filter is not None:
        ret = ret.loc[ret['agg'] == geo_filter, :]
    # Return only the core data to minimise the html size
    return json.loads(ret.loc[:, ['geometry']].to_json())

In [ ]:
#| hide
show_doc(SolomonGeo.get_geojson)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L187){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.get_geojson

>      SolomonGeo.get_geojson (geo_filter:str=None)

A getter method for the SolomonGeo class that returns a Geo JSON formatted dataset

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| geo_filter | str | None | Filters the geojson to the requested aggregation |
| **Returns** | **dict** |  | **Geo JSON formatted dataset** |

In [ ]:
#| export
@patch
def get_df(self:SolomonGeo, 
                geo_filter:str = None, # Filters the dataframe to the requested geography 
                var_filter:str = None, # Selects the desired column from the dataframe
                loc_filter:[str] = None, # Filters one of more locations
                # TODO remove hardcoding here?
                type_filter:str = 'Total', # Return either number of proportion
                agg_flag = False, # Whether to return the dataset aggregated for the given selection
               ) -> pd.DataFrame: # Pandas Dataframe containing population data
    '''
    A getter method for the SolomonGeo class that returns a pandas dataset containg
    the id variable and the total population variable. This is the minimal data required
    to display on the map. 
    '''
    ret = self.geo_df
    ret = ret.loc[ret['type'] == type_filter, :]
    # TODO check that filter is valid
    if geo_filter is not None:
        ret = ret.loc[ret['agg'] == geo_filter, :]
    # Return only the core data to minimise the html size
    names = ['geometry', 'id', 'agg', 'year', 'type']
    ret = ret.drop(columns = names)

    if loc_filter is not None:
        ret = ret.loc[ret.index.isin(loc_filter), :]

    # Keep only selected column if required
    if var_filter is not None:
        ret = ret.filter(items = [var_filter])
        
    return pd.DataFrame(ret)

In [ ]:
#| hide
show_doc(SolomonGeo.get_df)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L203){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo.get_df

>      SolomonGeo.get_df (geo_filter:str=None, var_filter:str=None,
>                         loc_filter:[<class'str'>]=None,
>                         type_filter:str='Total', agg_flag=False)

A getter method for the SolomonGeo class that returns a pandas dataset containg
the id variable and the total population variable. This is the minimal data required
to display on the map.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| geo_filter | str | None | Filters the dataframe to the requested geography |
| var_filter | str | None | Selects the desired column from the dataframe |
| loc_filter | [<class 'str'>] | None | Filters one of more locations |
| type_filter | str | Total | Return either number of proportion |
| agg_flag | bool | False | Whether to return the dataset aggregated for the given selection |
| **Returns** | **DataFrame** |  | **Pandas Dataframe containing population data** |

## Aggregate Dataframe
Returns an aggregate pandas series 

In [ ]:
#| export
@patch
def agg_df(self:SolomonGeo, 
                geo_filter:str = None, # Filters the dataframe to the requested geography 
                var_filter:str = None, # Selects the desired column from the dataframe
                loc_filter:[str] = None, # Filters one of more locations
                # TODO remove hardcoding here?
                type_filter:str = 'Total', # Return either number of proportion
               ) -> pd.Series: # Pandas data series containing aggregated and filtered data
    '''
    A getter method for the SolomonGeo class that calls get_df to get a spcific and then further 
    aggregates that dataset so that the proportion is the weighted proportion
    '''
    df = self.get_df(geo_filter, var_filter, loc_filter, type_filter)

    if type_filter == 'Total':
        df = df.sum()
    elif type_filter == 'Proportion':
        tot_df = self.get_df(geo_filter, var_filter, loc_filter, 'Total')
        df = df * tot_df
        df = df.sum() / tot_df.sum()
    else:
        raise ValueError('The type passed to the aggregate function must be one of the following: \'Total\', \'Proportion\'.')
    return df

In [ ]:
show_doc(agg_df)

---

[source](https://github.com/fastai/nbdev/blob/master/nbdev/showdoc.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### show_doc

>      show_doc (sym, renderer=None, name:str|None=None, title_level:int=3)

Show signature and docstring for `sym`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sym |  |  | Symbol to document |
| renderer | NoneType | None | Optional renderer (defaults to markdown) |
| name | str \| None | None | Optionally override displayed name of `sym` |
| title_level | int | 3 | Heading level to use for symbol name |

# Testing

In [ ]:
sol_geo = SolomonGeo.read_test()

In [ ]:
sol_geo.geo_df

,geometry,id,agg,year,type,Key Statistics: Total Households,Main source of household drinking water: metered SIWA drinking water,Main source of household drinking water: communal standpipe,Main source of household drinking water: private water tank,Main source of household drinking water: communal water tank,...,Household money received from remittances: 1000 - 1499 S.I. dollars,Household money received from remittances: more than 1500 S.I. dollars,Main source of household income in last 12 months: No income,Main source of household income in last 12 months: Wages Salary,Main source of household income in last 12 months: Own business,Main source of household income in last 12 months: Sale fish crop craft,Main source of household income in last 12 months: Land lease,Main source of household income in last 12 months: House rent,Main source of household income in last 12 months: Remittances,Main source of household income in last 12 months: Other source
geo_name,,,,,,,,,,,,,,,,,,,,,
Choiseul,"MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...",1,Province,2009,Total,4712,2.00,1416.00,1137.00,840.00,...,209.00,179.00,88.00,892.00,206.00,2443.00,6.00,16.00,206.00,855.00
Choiseul,"MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...",1,Province,2009,Proportion,100,0.04,30.05,24.13,17.83,...,4.44,3.80,1.87,18.93,4.37,51.85,0.13,0.34,4.37,18.15
Honiara,"MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...",10,Province,2009,Total,8981,6771.00,351.00,861.00,134.00,...,203.00,395.00,75.00,6984.00,783.00,393.00,54.00,179.00,48.00,465.00
Honiara,"MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...",10,Province,2009,Proportion,100,75.39,3.91,9.59,1.49,...,2.26,4.40,0.84,77.76,8.72,4.38,0.60,1.99,0.53,5.18
Western,"MULTIPOLYGON (((158.25148 -8.78861, 158.24885 ...",2,Province,2009,Total,13762,37.00,4151.00,4596.00,2426.00,...,530.00,478.00,182.00,3732.00,618.00,7518.00,8.00,48.00,410.00,1246.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vanikoro,"MULTIPOLYGON (((166.98326 -11.67945, 166.98258...",915,Ward,2009,Proportion,100,0.38,62.78,3.38,1.50,...,0.75,0.00,4.89,4.89,6.39,46.99,0.00,0.00,0.00,36.84
Tikopia,"MULTIPOLYGON (((168.84036 -12.28504, 168.82691...",916,Ward,2009,Total,262,0.00,255.00,1.00,4.00,...,10.00,4.00,39.00,19.00,2.00,87.00,0.00,0.00,26.00,89.00
Tikopia,"MULTIPOLYGON (((168.84036 -12.28504, 168.82691...",916,Ward,2009,Proportion,100,0.00,97.33,0.38,1.53,...,3.82,1.53,14.89,7.25,0.76,33.21,0.00,0.00,9.92,33.97


In [ ]:
sol_geo.locations

{'Province': Index(['Central', 'Choiseul', 'Guadalcanal', 'Honiara', 'Isabel',
        'Makira-Ulawa', 'Malaita', 'Rennell-Bell', 'Temotu', 'Western'],
       dtype='object', name='geo_name'),
 'Constituency': Index(['Auki-Langalanga', 'Baegu-Asifola', 'Central Guadalcanal',
        'Central Honiara', 'Central Kwara'ae', 'Central Makira', 'East AreAre',
        'East Choiseul', 'East Guadalcanal', 'East Honiara', 'East Kwaio',
        'East Makira', 'East Malaita', 'East-Central', 'Fataleka', 'Gao-Bugotu',
        'Gizo-Kolombangara', 'Hograno-Kia-Havulei', 'Lau-Mbaelela',
        'Malaita Outer Island', 'Marigne-Kokota', 'Marovo', 'Nggela',
        'North Guadalcanal', 'North Malaita', 'North New Georgia',
        'North Vella la Vella', 'North West Choiseul', 'North West Guadalcanl',
        'North-East', 'Ranongga-Simbo', 'Rennell-Bellona',
        'Savo-Russull Islands', 'Shortland Islands', 'Small Malaita',
        'South Choiseul', 'South Guadalcanal', 'South New Georgia-Rendova'

Save and Load

In [ ]:
sol_geo.save_pickle('/testData/')

In [ ]:
SolomonGeo.load_pickle('/testData/')

<__main__.SolomonGeo>

Test filtering of down to multiple locations

In [ ]:
#test_eq(sol_geo.get_df(geo_filter = 'constituency')['total_pop'].sum(), const_df['total_pop'].sum())
sol_geo.get_df(geo_filter = 'Constituency', loc_filter=['Nggela', 'East AreAre'])

,Key Statistics: Total Households,Main source of household drinking water: metered SIWA drinking water,Main source of household drinking water: communal standpipe,Main source of household drinking water: private water tank,Main source of household drinking water: communal water tank,Main source of household drinking water: well - protected,Main source of household drinking water: well - unprotected,Main source of household drinking water: river or stream,Main source of household drinking water: bottled water,Main source of household drinking water: other source of drinking water,...,Household money received from remittances: 1000 - 1499 S.I. dollars,Household money received from remittances: more than 1500 S.I. dollars,Main source of household income in last 12 months: No income,Main source of household income in last 12 months: Wages Salary,Main source of household income in last 12 months: Own business,Main source of household income in last 12 months: Sale fish crop craft,Main source of household income in last 12 months: Land lease,Main source of household income in last 12 months: House rent,Main source of household income in last 12 months: Remittances,Main source of household income in last 12 months: Other source
geo_name,,,,,,,,,,,,,,,,,,,,,
East AreAre,1105,1.0,815.0,12.0,9.0,1.0,3.0,223.0,0.0,41.0,...,41.0,15.0,37.0,89.0,101.0,650.0,0.0,2.0,31.0,195.0
Nggela,3315,26.0,1557.0,258.0,437.0,52.0,13.0,868.0,1.0,103.0,...,25.0,59.0,49.0,558.0,327.0,1878.0,0.0,4.0,75.0,424.0


In [ ]:
# TODO work out how to check save and load are the same as a test
# Might need to wreite and equality function

In [ ]:
testDF = sol_geo.geo_df

In [ ]:
testDF = testDF.rename(columns = {'geometry':'core: geometry', 'id':'core: id', 'agg':'core: agg', 'year':'core: year', 'type':'core: type'})


In [ ]:
cols = testDF.columns.str.extract(r'(.*): (.+)', expand=True)

In [ ]:
cols

,0,1
0,core,geometry
1,core,id
2,core,agg
3,core,year
4,core,type
...,...,...
66,Main source of household income in last 12 months,Sale fish crop craft
67,Main source of household income in last 12 months,Land lease
68,Main source of household income in last 12 months,House rent
69,Main source of household income in last 12 months,Remittances


In [ ]:
testDF.columns = pd.MultiIndex.from_arrays((cols[0], cols[1]))
testDF.columns.names = [None]*2

In [ ]:
testDF[('core')]['geometry']

geo_name
Choiseul    MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...
Choiseul    MULTIPOLYGON (((157.56463 -7.49272, 157.56005 ...
Honiara     MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...
Honiara     MULTIPOLYGON (((159.91806 -9.42379, 159.92354 ...
Western     MULTIPOLYGON (((158.25148 -8.78861, 158.24885 ...
                                  ...                        
Vanikoro    MULTIPOLYGON (((166.98326 -11.67945, 166.98258...
Tikopia     MULTIPOLYGON (((168.84036 -12.28504, 168.82691...
Tikopia     MULTIPOLYGON (((168.84036 -12.28504, 168.82691...
Neo         MULTIPOLYGON (((165.81001 -10.65326, 165.79693...
Neo         MULTIPOLYGON (((165.81001 -10.65326, 165.79693...
Name: geometry, Length: 486, dtype: geometry

In [ ]:
# TODO should do a count check for the geojson similar to this
sol_geo.geo_df[sol_geo.geo_df['agg'] == 'Constituency'].count()

geometry                                                                   100
id                                                                         100
agg                                                                        100
year                                                                       100
type                                                                       100
                                                                          ... 
Main source of household income in last 12 months: Sale fish crop craft    100
Main source of household income in last 12 months: Land lease              100
Main source of household income in last 12 months: House rent              100
Main source of household income in last 12 months: Remittances             100
Main source of household income in last 12 months: Other source            100
Length: 71, dtype: int64

# TODO check that all proportions are less than 0

In [ ]:
# TODO filter and test equality?
test = sol_geo.get_geojson(geo_filter = 'ward')

The sum after merging and filtering should equal the sum from the raw dataset.

In [ ]:
test_eq(sol_geo.agg_df(geo_filter = 'Constituency')['Key Statistics: Total Households'], 
        const_df.loc[const_df['type'] == 'Total'].loc[const_df['agg'] == 'Constituency']['Key Statistics: Total Households'].sum())

In [ ]:
sol_geo.agg_df(geo_filter = 'Constituency', var_filter='Key Statistics: Total Households')

Key Statistics: Total Households    91251
dtype: int64

In [ ]:
sol_geo.get_df(geo_filter = 'Constituency', var_filter='Key Statistics: Total Households', loc_filter=['West Guadalcanal']).values

array([[1671]])

In [ ]:
# TODO - test that there are enough rows - don't drop any during merging...
sol_geo.agg_df(geo_filter = 'Constituency', type_filter='Proportion')

Key Statistics: Total Households                                           100.000000
Main source of household drinking water: metered SIWA drinking water        64.958065
Main source of household drinking water: communal standpipe                 48.022223
Main source of household drinking water: private water tank                 29.904607
Main source of household drinking water: communal water tank                21.749937
                                                                              ...    
Main source of household income in last 12 months: Sale fish crop craft     52.345803
Main source of household income in last 12 months: Land lease                0.421429
Main source of household income in last 12 months: House rent                1.234090
Main source of household income in last 12 months: Remittances               3.970852
Main source of household income in last 12 months: Other source             27.073865
Length: 66, dtype: float64

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()